<a href="https://colab.research.google.com/github/jimni0/handson_ml/blob/main/Ch7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7장. 앙상블 학습과 랜덤 포레스트


* 일련의 예측기 (분류나 회귀 모델)로 부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측이 가능함.
* 일련의 예측기를 앙상블이라고 부르기 때문에 이를 **앙상블 학습**이라고 함. 앙상블 학습 알고리즘을 **앙상블 방법**이라고 함.

* 앙상블 방법의 예로 훈련 세트로부터 랜덤으로 각기 다른 서브셋을 만들어 일련의 결정 트리 분류기를 훈련시킬 수 있음. 개별 트리의 예측을 보아 가장 많은 선택을 받은 클래스를 앙상블의 예측으로 삼음.
* 결정 트리의 앙상블을 랜덤 포레스트라고 함.



## 7.1 투표 기반 분류기
* 좋은 분류기를 만드는 가장 간단한 방법은 각 분류기의 예측을 집계하는 것임. 가장 많은 표를 얻은 클래스가 앙상블의 예측이 됨.
* **직접 투표 분류기**: 다수결 추표로 정해지는 분류기
* 이 다수결 투표 분류기가 앙상블에 포함된 개별 분류기 중 가장 뛰어난 것보다도 정확도가 높은 경우가 많음. 사실 각 분류기가 **약한 분류기**(랜덤 추측보다 조금 더 높은 성능을 내는 분류기)일지라도 앙상블에 있는 약한 학습기가 충분하게 많고 다양하다면 앙상블은 (높은 정확도를 내는) **강한 학습기**가 될 수 있음.
* 앙상블 방법은 *예측기가 가능한 한 서로 독립적일 때* 최고의 성능을 발휘함. 다양한 분류기를 얻는 한 가지 방법은 각기 다른 알고리즘으로 학습시키는 것임. 이렇게 하면 매우 다른 종류의 오차를 만들 가능성이 높기 때문에 앙상블 모델의 정확도가 향상됨.

In [2]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

voting_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(random_state=42))
        ]
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

VotingClassifier를 훈련할 때 이 클래스는 모든 추정기를 복제해 복제된 추정기를 훈련함.

In [3]:
for name, clf in voting_clf.named_estimators_.items():
  print(name, '=', clf.score(X_test, y_test))

lr = 0.864
rf = 0.896
svc = 0.896


In [4]:
# 투표 기반 분류기는 predict() 메서드를 호출하면 직접 투표를 수행함.
# 예를 들어 이 투표 기반 분류기는 테스트 세트의 첫 번째 샘플에 대해 클래스 1을 예측하는데,
# 이는 세 분류기 중 두 분류기가 해당 클래스를 예측하기 때문임
voting_clf.predict(X_test[:1])

array([1])

In [5]:
[clf.predict(X_test[:1]) for clf in voting_clf.estimators_]

[array([1]), array([1]), array([0])]

In [6]:
# 테스트 세트에서 투표 기반 분류기의 성능을 살펴볼 것임
voting_clf.score(X_test, y_test)

0.912



*  **간접 투표**: 모든 분류기가 클래스의 확률을 예측할 수 있으면 (predict_proba() 메서드가 있다면) 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있음.
*   이 방식은 확률이 높은 투표에 비중을 더 두기 때문에 직접 투표 방식보다 성능이 높음.



In [7]:
# 이 방식을 사용하기 위해 투표 기반 분류기의 voting 매개변수를 "soft"로 바꾸고, 모든 분류기가 클래스의 확률을 추정할 수 있도록 하면 됨.
voting_clf.voting = "soft"
voting_clf.named_estimators["svc"].probability = True
voting_clf.fit(X_train, y_train)
voting_clf.score(X_test, y_test)

0.92

## 7.2 배깅과 페이스팅
* 다양한 분류기를 만드는 다른 방법은 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 랜덤으로 구성하여 분류기를 각기 다르게 학습시키는 것임.
* **배깅**: 훈련 세트에서 중복을 허용해 샘플링하는 방식
* **페이스팅**: 중복을 허용하지 않고 샘플링 하는 방식

* 모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 만듦.
* 집계 함수는 일반적으로 분류일 때는 **통계적 최빈값**, 회귀에서는 평균을 계산함.
* 개별 예측기는 원본 훈련 세트로 훈련시킨 것보다 훨씬 크게 편향되어 있지만 집계 함수를 통과하면 평향과 분산이 모두 감소함.
* 일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬 때와 비교해 편향은 비슷하지만 분산은 줄어듦.


### 7.2.1 사이킷런의 배깅과 페이스팅

In [10]:
# 결정 트리 분류기 500개의 앙상블을 훈련시키는 코드
# 배깅 같은 경우, 각 분류기가 훈련 세트에서 중복을 허용하기 때문에 랜덤으로 선택된 100개의 샘플로 훈련함.
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, n_jobs=-1, random_state=42) # 페이스팅을 사용하려면 bootstrap=False
    # n_jobs 매개변수는 사이킷런이 훈련과 예측에 사용할 CPU 코어 수를 지정.
    # -1로 지정하면 가용한 모든 코어를 사용함.

bag_clf.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1, random_state=42)

앙상블은 비슷한 편향에서 더 작은 분산을 만듦. 즉, 훈련 세트의 오차 수가 거의 동일하지만 결정 경계는 덜 불규칙함.

* 배깅은 각 예측기가 학습하는 서브셋에 다양성을 추가하기 때문에 배깅이 페이스팅보다 편향이 조금 더 높음.
* 다양성을 추가한다는 것은 예측기들의 상관관계를 줄이므로 앙상블의 분산이 줄어든다는 것을 의미함.
* 전반적으로 배깅이 더 나은 모델을 만들기 때문에 일반적으로 더 선호됨.

### 7.2.2 OOB 평가
* 배깅을 사용할 때, 어떤 샘플은 한 에측기를 위해 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 수도 있음.
* 이는 평균적으로 각 예측기에 훈련 샘플의 63% 정도만 샘플링된다는 것을 의미함.
* 선택되지 않은 나머지 37%를 OOB 샘플이라고 부름. (예측기마다 남겨진 37%는 모두 다름)
* 예측기가 훈련되는 동안에는 OOB 샘플을 사용하지 않기에 별도의 검증 세트를 사용하지 않고 OOB 샘플을 사용해 평가할 수 있음.
* 앙상블의 평가는 각 예측기의 OOB 평가를 평균하여 얻음.


In [11]:
# 사이킷런에서 BaggingClassifier를 만들 때 oob_score=True로 지정 시, 훈련이 끝난 후 자동으로 OOB 평가를 수행함.
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            oob_score=True, random_state=42)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.896

In [12]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.92

In [13]:
# OOB 샘플에 대한 결정 함수 값도 확인 가능함.
bag_clf.oob_decision_function_[:3] # 처음 3개의 샘플에 대한 확률

array([[0.32352941, 0.67647059],
       [0.3375    , 0.6625    ],
       [1.        , 0.        ]])

# 7.3 랜덤 패치와 랜덤 서브스페이스

* BaggingClassifier는 특성 샘플링도 지원함. 샘플링은 max_features, bootstrap_features 두 매개변수로 조절됨.
* max_samples, bootstrap과 동일한 방식으로 작동하지만 샘플이 아닌 특성에 대한 샘플링에 사용됨.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16,
                                 n_jobs=-1, random_state=42)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

In [15]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="sqrt", max_leaf_nodes=16),
    n_estimators=500, n_jobs=-1, random_state=42)

In [17]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=30,
    learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=30, random_state=42)

In [18]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3 * X[:, 0] ** 2 + 0.05 * np.random.randn(100)  # y = 3x² + Gaussian noise

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [20]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=43)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2, random_state=43)

In [21]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=44)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2, random_state=44)

In [22]:
X_new = np.array([[-0.4], [0.], [0.5]])
sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

array([0.49484029, 0.04021166, 0.75026781])

In [23]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3,
                                 learning_rate=1.0, random_state=42)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=42)

In [24]:
gbrt_best = GradientBoostingRegressor(
    max_depth=2, learning_rate=0.05, n_estimators=500,
    n_iter_no_change=10, random_state=42)
gbrt_best.fit(X, y)

GradientBoostingRegressor(learning_rate=0.05, max_depth=2, n_estimators=500,
                          n_iter_no_change=10, random_state=42)

In [25]:
gbrt_best.n_estimators_

92

In [27]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder

hgb_reg = make_pipeline(
    make_column_transformer((OrdinalEncoder(), ["ocean_proximity"]),
                            remainder="passthrough"),
    HistGradientBoostingRegressor(categorical_features=[0], random_state=42)
)

In [28]:
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(probability=True, random_state=42))
    ],
    final_estimator=RandomForestClassifier(random_state=43),
    cv=5  # 교차 검증 폴드 수
)
stacking_clf.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(random_state=42)),
                               ('rf', RandomForestClassifier(random_state=42)),
                               ('svc', SVC(probability=True, random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=43))

In [29]:
stacking_clf.score(X_test, y_test)

0.928